# [第4章: 形態素解析](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch4)
夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

In [8]:
!ls ../Corpus/neko.txt

../Corpus/neko.txt


In [33]:
import MeCab
import re

In [11]:
with open("../Corpus/neko.txt") as neko, open("Output/neko.txt.mecab", mode="w") as neko_mecab:
    tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    sentences = neko.read().replace("　", "").replace("\n", "")
    neko_mecab.write(tagger.parse(sentences))

In [12]:
!ls -l Output/neko.txt.mecab

-rw-r--r-- 1 ke1dy ke1dy 11353686 Mar 22 10:06 Output/neko.txt.mecab


## 30. 形態素解析結果の読み込み
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [19]:
result = []
with open("Output/neko.txt.mecab") as neko_mecab:
    for sentence in neko_mecab:
        if sentence == "EOS\n" or sentence == "\n":
            continue

        # 表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
        morpheme = {}
        foo = sentence.split("\t")
        bar = foo[1].split(",")
        morpheme["surface"] = foo[0]
        morpheme["base"] = bar[6]
        morpheme["pos"] = bar[0]
        morpheme["pos1"] = bar[1]
        result.append(morpheme)

In [20]:
result[10]

{'surface': '生れ', 'base': '生れる', 'pos': '動詞', 'pos1': '自立'}

## 31. 動詞
動詞の表層形をすべて抽出せよ．

In [26]:
surfaces_verb = set()
for morpheme in result:
    if morpheme["pos"] == "動詞":
        surfaces_verb.add(morpheme["surface"])

surfaces_verb

{'嫌っ',
 'おっしゃっ',
 'よる',
 '造ろ',
 'やり込める',
 '引きずっ',
 '吹か',
 'しつけ',
 '持ち直す',
 '呑み込ん',
 '驚かす',
 '打っ',
 'あきらめる',
 '掠め',
 '曲っ',
 '懸かり',
 '削っ',
 '捕え',
 'もがい',
 '通じ',
 '乗りゃ',
 'つづか',
 '侵し',
 'しむ',
 '送',
 '忍ば',
 '使いこなせ',
 'きっ',
 '拝する',
 '届く',
 '恐れ入ら',
 'かぎっ',
 '見張っ',
 '書き散らし',
 '作っ',
 '逃げ延び',
 'しゃくい',
 '切る',
 '変え',
 '洗う',
 '充ち',
 '休ん',
 '込めん',
 '見付かる',
 'どっ',
 'かたちづくっ',
 '叫ん',
 '受け',
 '振り落とそ',
 '睨め',
 '重んぜ',
 '滑る',
 '吐か',
 '捕れん',
 '濁っ',
 '恐れ入り',
 '見付け',
 '仕ろ',
 '描い',
 'あつかっ',
 '過ぎる',
 '応ずる',
 '見つかる',
 'ひるん',
 'のけ',
 '制する',
 '登り',
 '立つ',
 '払っ',
 '言っ',
 '失わ',
 '絞め',
 '気が付い',
 '致し',
 '返っ',
 '有ん',
 '費やさ',
 '達せ',
 '曲がり',
 '窶し',
 'つき',
 '見舞う',
 '制さ',
 '写っ',
 '企て',
 '恐れ',
 '納め',
 '考え込ん',
 '併せ',
 '見渡す',
 '分る',
 '悟っ',
 'しなび',
 '従え',
 'はなち',
 '放さ',
 'がたつく',
 '譲っ',
 '繰返す',
 '掻い込ん',
 'とまり',
 '塗っ',
 '古ぼけ',
 'なれる',
 '破ら',
 '居り',
 '行ける',
 'とおっ',
 'はみ出し',
 '余っ',
 '抛り出し',
 '申し聞ける',
 'いじっ',
 '差し引か',
 '透かし',
 'する',
 'くべる',
 '歌っ',
 '伺っ',
 '書き入れ',
 '処する',
 'ゆ',
 '驚い',
 '蒔き',
 '捕れる',
 '滅する',
 'やれる',
 '出直そ',
 '吹き払い',

## 32. 動詞の原形
動詞の原形をすべて抽出せよ．

In [27]:
bases_verb = set()
for morpheme in result:
    if morpheme["pos"] == "動詞":
        bases_verb.add(morpheme["base"])

bases_verb

{'吸取る',
 'よる',
 'やり込める',
 '祟る',
 '思い込む',
 '持ち直す',
 'つかむ',
 '驚かす',
 '冠する',
 'あきらめる',
 '備わる',
 '失す',
 '慕う',
 'つれる',
 'こする',
 '遣わす',
 '見やる',
 '儲かる',
 '印せる',
 'いざる',
 'しむ',
 'せめる',
 '届く',
 '拝する',
 '受け合える',
 '踏み出せる',
 '解せる',
 '切る',
 '生じる',
 '荒立てる',
 '怪しむ',
 '洗う',
 '見付かる',
 '滑る',
 '這出る',
 '過ぎる',
 '応ずる',
 '見つかる',
 '制する',
 '立つ',
 '写す',
 '持てる',
 '振り翳す',
 'すり抜ける',
 'なやむ',
 '見舞う',
 'こうる',
 '揉む',
 '流れ込む',
 'がたつく',
 '見渡す',
 '分る',
 'ぬかる',
 '服せる',
 '繰返す',
 '終える',
 'なれる',
 '行ける',
 '押し付ける',
 '狼狽える',
 '積む',
 '申し聞ける',
 '塗る',
 '通せる',
 '恥づ',
 'する',
 'くべる',
 '競う',
 '演じる',
 '待ち構える',
 '打ち抜く',
 '滑らす',
 '処する',
 '手伝う',
 '捕れる',
 'やれる',
 '滅する',
 'いう',
 '犯す',
 '清める',
 'あく',
 '用いる',
 '起つ',
 '申し込む',
 '迎える',
 '入り乱れる',
 '下す',
 '押しかける',
 '懸かる',
 '揚げる',
 '起る',
 '延す',
 'しつくす',
 '停る',
 '行き詰る',
 '喜ぶ',
 '取り乱す',
 '浸る',
 '下せる',
 '云う',
 '於く',
 '分ける',
 '合う',
 '降り込む',
 '取り合せる',
 '詫び入る',
 'のぼる',
 'まかり間違う',
 '誂える',
 '添える',
 '祭る',
 '敗れる',
 'さる',
 '掠める',
 '擲つ',
 '澄ます',
 '振り向く',
 '窮する',
 '存じる',
 '遂げる',
 'くるまる',
 '譲る',
 'ふさぐ',
 'やっつける'

## 33. サ変名詞
サ変接続の名詞をすべて抽出せよ．

In [31]:
surfaces_noun_sa = set()
for morpheme in result:
    if morpheme["pos"] == "名詞" and morpheme["pos1"] == "サ変接続":
        surfaces_noun_sa.add(morpheme["surface"])

surfaces_noun_sa

{'入会',
 '逸出',
 '全滅',
 'ちゃ事',
 '心配',
 '一周',
 '休養',
 '遠で',
 '濫用',
 '目撃',
 '忠告',
 '覚悟',
 '罵倒',
 '敬意',
 '独創',
 'どんと',
 'お待ち',
 '合併',
 '超絶',
 '芝居',
 '咳払',
 '深入り',
 '反覆',
 '矯正',
 '立見',
 '両立',
 '在宿',
 '所有',
 '噂',
 '洗髪',
 '奉賀',
 '執着',
 '凝結',
 '愛読',
 '迂回',
 '勧誘',
 '拷問',
 '充満',
 '左右',
 '循環',
 '対坐',
 '論証',
 '来着',
 'ろか',
 '失笑',
 '察し',
 '苦労',
 '釘付',
 '容赦',
 'こせこせ',
 '写生',
 '清聴',
 '右往左往',
 '度外',
 '嘔吐',
 '返報',
 '出勤',
 '返礼',
 '邂逅',
 '厚遇',
 '嘆願',
 '当惑',
 '転化',
 '鼓吹',
 '脱却',
 '晴々',
 '口論',
 '歓迎',
 '用意',
 '通暁',
 'てはい',
 '辞儀',
 '長生き',
 'はいらん',
 '感服',
 '狂',
 '自称',
 '雑居',
 '恐く',
 '勘定',
 '在天',
 '蚕食',
 '朗吟',
 '試験',
 '理解',
 '彫刻',
 '逍遥',
 '捏造',
 'らく',
 '和かい',
 '演習',
 '増加',
 '狼狽',
 '尊敬',
 '歓喜',
 '膨脹',
 '発覚',
 '頓挫',
 '祈念',
 '問答',
 '起因',
 '苦心',
 '自乗',
 '及第',
 '専断',
 '遂行',
 '満腹',
 '黙読',
 '専攻',
 '入籍',
 '手分け',
 '生活',
 '開化',
 '全快',
 '宙返り',
 '仰天',
 '幽閉',
 '陳列',
 '按腹',
 '病気',
 '投函',
 '消毒',
 '満足',
 '恐悦',
 '威嚇',
 'そうだ',
 '察',
 '勤務',
 '後進',
 '想見',
 '伏',
 '恋着',
 '説明',
 '契約',
 '留守',
 '験',
 '催促',
 '忘却',
 '粘着',
 'せいせい',
 '衝突',


## 34. 「AのB」
2つの名詞が「の」で連結されている名詞句を抽出せよ．

In [32]:
bases_noun = set()
for morpheme in result:
    if morpheme["pos"] == "名詞":
        bases_noun.add(morpheme["base"])

bases_noun

{'定石',
 '胡',
 '真先',
 '後ろ向き',
 '目撃',
 '敏',
 '先君',
 '無才',
 '花曇り',
 '論理',
 '傷寒論',
 '始まったな',
 'ホヤ',
 '栄誉',
 '酒屋',
 '近火',
 '一坪',
 'そん',
 '猫なで声',
 '警察',
 '仏師',
 '山高帽',
 '容赦',
 '返礼',
 '名残り',
 '剣突',
 '教壇',
 '金箔',
 '棒組',
 '雌',
 '葉落ち',
 '短',
 '機微',
 '長生き',
 '新聞',
 'セン',
 '匆々',
 '椅子',
 '顛末',
 '保存会',
 '貴人',
 '日本派',
 '猫のように',
 '物笑い',
 '山の上',
 '矯風会',
 '卒業の日',
 'ずくめ',
 '殿',
 '下界',
 '倍',
 '構造',
 '城下',
 '按腹',
 '己',
 '軽率',
 '薯',
 '察',
 '恐れ',
 'すててこ',
 '人民',
 '戦役',
 '生命',
 '日間',
 '衆',
 '伝説',
 '垢',
 '代言',
 '識別',
 '馬肉',
 '告朔',
 'スリ',
 '死体',
 'あんまり',
 'ZOLA',
 '蛇蝎のごとく',
 '公言',
 '無駄骨',
 '泉岳寺',
 '真人間',
 '木の葉',
 '寝起き',
 '苦手',
 '日本',
 '甕',
 '納',
 '何',
 '投書',
 'お客',
 '要件',
 '悲鳴',
 '横行',
 '肖像',
 '個',
 'いつまでも',
 '座布団',
 '一飯君恩',
 '水泡',
 '花弁',
 '廃絶',
 '換言',
 '千古',
 '旧友',
 '物心',
 '物',
 '全区',
 '灯影',
 '臓腑',
 '日夜',
 'だだっ子',
 '纏綿',
 '短日月',
 '雑巾',
 '諾否',
 'たて',
 '従前',
 '皮',
 '無遠慮',
 '即座',
 '同族',
 '品川',
 '達磨',
 '僕は君',
 '夏蜜柑',
 '辛苦',
 '物色',
 '粗末',
 '平気',
 '親愛',
 '活版',
 'ご存じ',
 '奥歯',
 '吸い殻',
 '堅気',
 '御主人',
 '後ろ',
 '不恰好',
 '富

In [35]:
noun_no_noun = set()
for morpheme in result:
    surface = morpheme["surface"]
    if "の" not in surface:
        continue
    match = re.search(r"(.*)の(.*)", surface)
    if match.group(1) in bases_noun and match.group(2) in bases_noun:
        noun_no_noun.add(surface)

noun_no_noun

{'うちの猫',
 'このまま',
 'この世',
 'この声',
 'この女',
 'この辺',
 'この道',
 'この間',
 'そのうち',
 'そのため',
 'そのほか',
 'そのまま',
 'その他',
 'その後',
 'その昔',
 'その歌',
 'その物',
 'その間',
 '一杯のビール',
 '世の中',
 '中の間',
 '亀の甲',
 '二の腕',
 '二人の女',
 '人間の声',
 '今日の天気',
 '今日の詩',
 '化の皮',
 '卒業の日',
 '君の名前',
 '君の国',
 '君の詩',
 '君の話',
 '夏の夜',
 '天の河',
 '奥の間',
 '女の子',
 '宵の口',
 '尊敬の念',
 '山の上',
 '山の芋',
 '岩の上',
 '年の功',
 '床の間',
 '思いのほか',
 '日の出',
 '春の日',
 '最後の手段',
 '月の光',
 '木の葉',
 '本来の面目',
 '松の木',
 '板の間',
 '次の間',
 '気の毒',
 '水の上',
 '池の上',
 '火の手',
 '火の車',
 '猫の恋',
 '猫の額',
 '男の子',
 '緑の黒髪',
 '腋の下',
 '腹の中',
 '自殺の方法',
 '茶の木',
 '茶の間',
 '茶碗の中',
 '血の道',
 '袖の下',
 '身の丈',
 '身の上',
 '身の毛',
 '香の物',
 '馬の骨'}

## 35. 名詞の連接
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [38]:
concats_noun = set()

idx = 0
while idx < len(result):
    if result[idx]["pos"] != "名詞" or result[idx+1]["pos"] != "名詞":
        idx += 1
        continue

    concat_noun = result[idx]["surface"] + result[idx+1]["surface"]
    idx += 2
    while idx < len(result) and result[idx]["pos"] == "名詞":
        concat_noun += result[idx]["surface"]
        idx += 1
    concats_noun.add(concat_noun)

concats_noun

{'東風子',
 'その後野良',
 '其明日',
 '風中',
 '鼻論',
 '全く実業家',
 '世間的',
 '毎日毎日',
 '大方虫',
 '喰べて',
 '文盲漢',
 'さ加減',
 '云えん事',
 '応無所住而生其心',
 '警視庁刑事巡査吉田虎蔵',
 '展覧場',
 'もし相手',
 '僕なん',
 '角地面',
 '伯父さん将軍家',
 '女連れ',
 '迷亭君いつ',
 '仏様',
 'わる所',
 '是非元',
 '晩鼠',
 '斃す事',
 '何十',
 '大変東風',
 'ようやく自覚',
 '君シャンパン',
 '垣一重',
 '名人理学士水島寒月',
 'いくら自分',
 '墨堤',
 '東洋流',
 '一週三度',
 '吾人戦勝国',
 '大事そう',
 '一般猫児',
 '出掛',
 '文学博士',
 '残ってる三冊',
 'おらあまた胃',
 '通り風呂場',
 '自然その物',
 '晩書斎',
 '四十年後',
 '低い声',
 '候節',
 '扇使い',
 '金田令夫人',
 '諸君公徳',
 '全く果報者',
 '三ダース',
 '学問上',
 '一塊りなげ',
 '四十分',
 '癪に障る事',
 '君シャーレマン',
 '彼等希臘人',
 '五十円',
 '寸断寸断',
 '書冊出版',
 '当世紳士流',
 '道具類',
 '芸術的良心',
 '私の',
 'その後鼻',
 '金善方',
 '朗読家',
 '力学上',
 'あの男の',
 '普通一般',
 '上の者',
 '長い名',
 '宇宙的',
 '風教上',
 '壊わし',
 '五行ごと',
 '大概逆上',
 '力味',
 '全く苦沙弥',
 '神経病',
 '得て恭謙',
 '先刻一',
 '候間',
 '偶然君',
 '黒縮緬',
 '不平なんだい',
 '細帯',
 '当人自身',
 '活画',
 '候もの',
 '三十本',
 '禅機',
 '盲猫',
 '差し合',
 'んざどうだって',
 '人間以外',
 '御客様',
 'きっと人',
 '浴場全体',
 'いくら中学校',
 '赤毛布',
 'もし善意',
 'めたる稀代',
 'うちいずれ',
 '草廬',
 '今日中学程度以上',
 '掛念',
 '何疋',
 '自ら任ずる',


## 36. 単語の出現頻度
文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## 37. 頻度上位10語
出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 38. ヒストグラム
単語の出現頻度のヒストグラム（横軸に出現頻度，縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け．

## 39. Zipfの法則
単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．